
https://en.wikipedia.org/wiki/Image_scaling#Algorithms

to do :
- own validation set
- resize images transformer (Bilinear done!)
- make noise transformer
-

In [1]:
# Let's load and process the dataset
import numpy as np
from fuel.datasets.dogs_vs_cats import DogsVsCats
train = DogsVsCats(('train',), subset=slice(0, 20000))
from fuel.streams import DataStream
from fuel.schemes import ShuffledScheme
from fuel.transformers.image import RandomFixedSizeCrop
from fuel.transformers import Flatten

# Load the training set
train = DogsVsCats(('train',), subset=slice(0, 100))

# We now create a "stream" over the dataset which will return shuffled batches
# of size 128. Using the DataStream.default_stream constructor will turn our
# 8-bit images into floating-point decimals in [0, 1].
stream = DataStream.default_stream(
    train,
    iteration_scheme=ShuffledScheme(train.num_examples, 128)
)


In [ ]:
#This whole cell is copy-paste from https://ift6266h16.wordpress.com/class-project/getting-started/


from fuel.streams import DataStream
from fuel.schemes import ShuffledScheme
from fuel.transformers.image import RandomFixedSizeCrop
from fuel.transformers import Flatten

# Load the training set
train = DogsVsCats(('train',), subset=slice(0, 20000))

# We now create a "stream" over the dataset which will return shuffled batches
# of size 128. Using the DataStream.default_stream constructor will turn our
# 8-bit images into floating-point decimals in [0, 1].
stream = DataStream.default_stream(
    train,
    iteration_scheme=ShuffledScheme(train.num_examples, 128)
)

# Our images are of different sizes, so we'll use a Fuel transformer
# to take random crops of size (32 x 32) from each image
cropped_stream = RandomFixedSizeCrop(
    stream, (32, 32), which_sources=('image_features',))

# We'll use a simple MLP, so we need to flatten the images
# from (channel, width, height) to simply (features,)
flattened_stream = Flatten(
    cropped_stream, which_sources=('image_features',))


#----------------
# Create the Theano MLP
import theano
from theano import tensor
import numpy

X = tensor.matrix('image_features')
T = tensor.lmatrix('targets')

W = theano.shared(
    numpy.random.uniform(low=-0.01, high=0.01, size=(3072, 500)), 'W')
b = theano.shared(numpy.zeros(500))
V = theano.shared(
    numpy.random.uniform(low=-0.01, high=0.01, size=(500, 2)), 'V')
c = theano.shared(numpy.zeros(2))
params = [W, b, V, c]

H = tensor.nnet.sigmoid(tensor.dot(X, W) + b)
Y = tensor.nnet.softmax(tensor.dot(H, V) + c)

loss = tensor.nnet.categorical_crossentropy(Y, T.flatten()).mean()

# Use Blocks to train this network
from blocks.algorithms import GradientDescent, Scale
from blocks.extensions import Printing
from blocks.extensions.monitoring import TrainingDataMonitoring
from blocks.main_loop import MainLoop

algorithm = GradientDescent(cost=loss, parameters=params,
                            step_rule=Scale(learning_rate=0.1))

# We want to monitor the cost as we train
loss.name = 'loss'
extensions = [TrainingDataMonitoring([loss], every_n_batches=1),
              Printing(every_n_batches=1)]

main_loop = MainLoop(data_stream=flattened_stream, algorithm=algorithm,
                     extensions=extensions)
#main_loop.run()


In [1]:
from __future__ import division
import math

import numpy

from fuel.transformers import ExpectsAxisLabels, SourcewiseTransformer

class BilinearRescale(SourcewiseTransformer, ExpectsAxisLabels):
    """Resize an image to a fixed window size. Use bilinear interpolation with 4-relative nearest neighbors.
    Parameters
    ----------
    data_stream : :class:`AbstractDataStream`
        The data stream to wrap.
    window_shape : tuple
        The `(height, width)` tuple representing the size of the output
        window.
    Notes
    -----
    This transformer expects to act on stream sources which provide one of
     * Single images represented as 3-dimensional ndarrays, with layout
       `(channel, height, width)`.
     * Batches of images represented as lists of 3-dimensional ndarrays,
       possibly of different shapes (i.e. images of differing
       heights/widths).
     * Batches of images represented as 4-dimensional ndarrays, with
       layout `(batch, channel, height, width)`.
    The format of the stream will be un-altered, i.e. if lists are
    yielded by `data_stream` then lists will be yielded by this
    transformer.
    """
    def __init__(self, data_stream, image_shape, **kwargs):
        self.image_shape = image_shape
        kwargs.setdefault('produces_examples', data_stream.produces_examples)
        kwargs.setdefault('axis_labels', data_stream.axis_labels)
        super(BilinearRescale, self).__init__(data_stream, **kwargs)

    def transform_source_batch(self, source, source_name):
        self.verify_axis_labels(('batch', 'channel', 'height', 'width'),
                                self.data_stream.axis_labels[source_name],
                                source_name)
        height, width = self.image_shape
        print(source.shape)
        if isinstance(source, numpy.ndarray) and source.ndim == 4:
            # Hardcoded assumption of (batch, channels, height, width).
            # This is what the fast Cython code supports.
            raise Exception
        
        elif all(isinstance(b, numpy.ndarray) and b.ndim == 3 for b in source):
            return [self.transform_source_example(im, source_name)
                    for im in source]
        else:
            raise ValueError("uninterpretable batch format; expected a list "
                             "of arrays with ndim = 3, or an array with "
                             "ndim = 4")

    def transform_source_example(self, example, source_name):
        self.verify_axis_labels(('channel', 'height', 'width'),
                                self.data_stream.axis_labels[source_name],
                                source_name)
        height, width = self.image_shape
        nb_channel = example.shape[0] #That line could be replace by something more elegant.
        
        if not isinstance(example, numpy.ndarray) or example.ndim != 3:
            raise ValueError("uninterpretable example format; expected "
                             "ndarray with ndim = 3")
        image_height, image_width = example.shape[1:]
        rescale_height, rescale_width = (image_height-1)/(height-1), (image_width-1)/(width-1)
        
        rescaled_image = np.zeros((nb_channel,height,width))
        #Might do a cleaner version eventually
        #print(example[0][0])
        #print(example.shape)
        
        for i,j in product(range(height), range(width)):
            x, y  = i*rescale_height, j * rescale_width
            x1 = np.array([math.floor(x), math.ceil(x)],dtype=np.intp)
            y1 = np.array([math.floor(y), math.ceil(y)],dtype=np.intp)
            dx,dy = x - x1[0], y - y1[0]
            del_x, del_y = x1[1]-x1[0], y1[1]-y1[0]
            
            #Stupid patching for float approximation induced problem
            if x1[1] == image_height:
                x1[1] -= 1
            if y1[1] == image_width:
                y1[1] -= 1
                
            if not x1[0] == x1[1] and not y1[0] == y1[1] :
                for c in range(nb_channel):
                    xy1,xy2 = example[c][np.ix_(x1,y1)]
                    x1y1,x2y1,x1y2,x2y2 = xy1[0],xy1[1],xy2[0],xy2[1]
                    rescaled_image[c,i,j] = (x2y1-x1y1)*dx/del_x + (x1y2 -x1y1) *dy/del_y + (x1y1 + x2y2 - x2y1 - x1y2) *dx/del_x*dy/del_y + x1y1
            elif x1[0] == x1[1] and y1[0] == y1[1]:
                rescaled_image[:,i,j] = example[:,i,j]
            else:
                if y1[0] == y1[1]:
                    for c in range(nb_channel):
                        x1y,x2y = example[c][np.ix_(x1,y1)]
                        x1y,x2y = x1y[0], x1y[1]
                        rescaled_image[c,i,j] = (x2y-x1y)*dx/del_x + x1y
                else:
                    for c in range(nb_channel):
                        xy1,xy2 = example[c][np.ix_(x1,y1)]
                        xy1,xy2 = xy1[0], xy1[1]
                        rescaled_image[c,i,j] = (xy2 - xy1) * dy/del_y + xy1
            
        return rescaled_image
    


In [3]:
s0,s1 = 50,50
nb_hidden = 600

x = BilinearRescale(stream, (s0,s1), which_sources=('image_features',))
#    stream, (32, 32), which_sources=('image_features',))

flattened_stream = Flatten(
    x, which_sources=('image_features',))

In [4]:
import theano
from itertools import product
from theano import tensor
import numpy
import numpy as np
X = tensor.matrix('image_features')
T = tensor.lmatrix('targets')

W = theano.shared(
    numpy.random.uniform(low=-0.01, high=0.01, size=(s0*s1*3, nb_hidden)), 'W')
b = theano.shared(numpy.zeros(nb_hidden))
V = theano.shared(
    numpy.random.uniform(low=-0.01, high=0.01, size=(nb_hidden, 2)), 'V')
c = theano.shared(numpy.zeros(2))
params = [W, b, V, c]

H = tensor.nnet.sigmoid(tensor.dot(X, W) + b)
Y = tensor.nnet.softmax(tensor.dot(H, V) + c)

loss = tensor.nnet.categorical_crossentropy(Y, T.flatten()).mean()

# Use Blocks to train this network
from blocks.algorithms import GradientDescent, Scale
from blocks.extensions import Printing
from blocks.extensions.monitoring import TrainingDataMonitoring
from blocks.bricks.cost import CategoricalCrossEntropy,SquaredError,MisclassificationRate
from blocks.main_loop import MainLoop


classification_error= MisclassificationRate().apply(np.argmax(Y),T)

algorithm = GradientDescent(cost=loss, parameters=params,
                            step_rule=Scale(learning_rate=0.1))

# We want to monitor the cost as we train
loss.name = 'loss'
extensions = [TrainingDataMonitoring([loss, classification_error], every_n_batches=1),
              Printing(every_n_batches=1)]

main_loop = MainLoop(data_stream=flattened_stream, algorithm=algorithm,
                     extensions=extensions)
main_loop.run()


-------------------------------------------------------------------------------
BEFORE FIRST EPOCH
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: False
	 epoch_started: True
	 epochs_done: 0
	 iterations_done: 0
	 received_first_batch: False
	 resumed_from: None
	 training_started: True
Log records from the iteration 0:

(100,)

-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: False
	 epoch_started: True
	 epochs_done: 0
	 iterations_done: 1
	 received_first_batch: True
	 resumed_from: None
	 training_started: True
Log records from the iteration 1:
	 loss: 0.692335153836
	 misclassificationrate_apply_error_rate: 1.0


-----------------------------------------------------------------------


Blocks will complete this epoch of training and run extensions before exiting. If you do not want to complete this epoch, press CTRL + C again to stop training after the current batch.




-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: True
	 epoch_started: True
	 epochs_done: 21
	 iterations_done: 22
	 received_first_batch: True
	 resumed_from: None
	 training_started: True
Log records from the iteration 22:
	 loss: 0.685569927051
	 misclassificationrate_apply_error_rate: 1.0


-------------------------------------------------------------------------------
AFTER ANOTHER EPOCH
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: True
	 epoch_started: False
	 epochs_done: 22
	 iterations_done: 22
	 received_first_batch: True
	 resumed_from: None
	 training_started: True
Log records from the iteration 22:
	 loss: 0.685569927051
	 misclassificationrate_apply_error_rate: 1.0


---

In [ ]:
%debug

In [ ]:
print(image_width,image_height)